In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os
from PIL import Image
import matplotlib.pyplot as plt
from collections import Counter

print("ok")

ok


In [13]:
df = pd.read_csv('pokemon.csv')
df

,Name,Type1,Type2,Evolution
0,bulbasaur,Grass,Poison,ivysaur
1,ivysaur,Grass,Poison,venusaur
2,venusaur,Grass,Poison,NaN
3,charmander,Fire,NaN,charmeleon
4,charmeleon,Fire,NaN,charizard
...,...,...,...,...
804,stakataka,Rock,Steel,NaN
805,blacephalon,Fire,Ghost,NaN
806,zeraora,Electric,NaN,NaN
807,meltan,Steel,NaN,NaN


In [9]:
main_types = ['Water', 'Fire', 'Grass', 'Electric', 'Psychic', 'Normal']
df_filtered = df[df['Type1'].isin(main_types)].copy()

print(f"Filtered to {len(df_filtered)} Pokémon from main 6 types")

Filtered to 443 Pokémon from main 6 types


In [10]:

# লেবেল এনকোডিং
label_encoder = LabelEncoder()
df_filtered['Type_encoded'] = label_encoder.fit_transform(df_filtered['Type1'])

print(f"Training on {len(main_types)} types: {main_types}")


Training on 6 types: ['Water', 'Fire', 'Grass', 'Electric', 'Psychic', 'Normal']


In [14]:
# ইমেজ ডিরেক্টরি
image_dir = 'pokemon_images'


pokemon_images


In [12]:
# ইমেজ লোড করার ফাংশন
def load_images_and_labels(df, image_dir, image_size=(128, 128)):
    images = []
    labels = []
    missing_images = []
    
    for idx, row in df.iterrows():
        pokemon_name = row['Name']
        possible_extensions = ['.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG']
        img_path = None
        
        for ext in possible_extensions:
            temp_path = os.path.join(image_dir, pokemon_name + ext)
            if os.path.exists(temp_path):
                img_path = temp_path
                break
        
        if img_path is None:
            missing_images.append(pokemon_name)
            continue
            
        try:
            img = Image.open(img_path)
            
            if img.mode == 'RGBA':
                img = img.convert('RGB')
            
            img = img.resize(image_size)
            img_array = np.array(img)
            
            img_array = img_array.astype('float32') / 255.0
            
            images.append(img_array)
            labels.append(row['Type_encoded'])
            
        except Exception as e:
            print(f"Error loading image {img_path}: {str(e)}")
            missing_images.append(pokemon_name)
    
    print(f"Successfully loaded {len(images)} images")
    print(f"Missing {len(missing_images)} images")
    
    return np.array(images), np.array(labels), missing_images

# ডেটা লোড
print("Loading images...")
X, y, missing_images = load_images_and_labels(df_filtered, image_dir)

if len(X) == 0:
    print("No images loaded!")
else:
    print(f"Final dataset: {len(X)} images")
    
    # ক্লাস ডিস্ট্রিবিউশন
    class_distribution = Counter(y)
    print("\nClass distribution:")
    for cls, count in class_distribution.items():
        type_name = label_encoder.inverse_transform([cls])[0]
        print(f"  {type_name}: {count} samples")
    
    # ট্রেন-টেস্ট স্প্লিট
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"\nTraining set: {X_train.shape}")
    print(f"Test set: {X_test.shape}")
    
    num_classes = len(main_types)
    
    # খুব SIMPLE মডেল - ছোট ডেটাসেটের জন্য
    model = keras.Sequential([
        # প্রথম কনভোল্যুশন লেয়ার
        layers.Conv2D(16, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        layers.MaxPooling2D((2, 2)),
        
        # দ্বিতীয় কনভোল্যুশন লেয়ার  
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # তৃতীয় কনভোল্যুশন লেয়ার
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # ফ্ল্যাটেন এবং ডেনস লেয়ার
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),  # কম ড্রপআউট
        layers.Dense(num_classes, activation='softmax')
    ])
    
    # মডেল কম্পাইল - কম learning rate
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.summary()
    
    # সরল ট্রেনিং - ডেটা অগমেন্টেশন ছাড়া
    print("\nTraining simple model...")
    
    history = model.fit(
        X_train, y_train,
        batch_size=8,  # ছোট ব্যাচ সাইজ
        epochs=50,
        validation_data=(X_test, y_test),
        callbacks=[
            keras.callbacks.EarlyStopping(
                patience=20, 
                restore_best_weights=True,
                monitor='val_accuracy'
            )
        ],
        verbose=1
    )
    
    # ইভ্যালুয়েশন
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"\nTest Accuracy: {test_accuracy:.4f}")
    print(f"Test Loss: {test_loss:.4f}")
    
    # মডেল সেভ
    model.save('pokemon_simple_model.h5')
    
    import joblib
    joblib.dump(label_encoder, 'label_encoder_simple.pkl')
    print("Simple model saved!")
    
    # প্রেডিকশন ফাংশন
    def predict_pokemon_type_simple(image_path, model, label_encoder):
        img = Image.open(image_path)
        img = img.resize((128, 128))
        
        if img.mode == 'RGBA':
            img = img.convert('RGB')
            
        img_array = np.array(img).astype('float32') / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        
        prediction = model.predict(img_array, verbose=0)
        predicted_class = np.argmax(prediction, axis=1)
        confidence = np.max(prediction, axis=1)
        
        predicted_type = label_encoder.inverse_transform(predicted_class)
        
        # সব ক্লাসের probability দেখানো
        print("\nAll predictions:")
        for i, prob in enumerate(prediction[0]):
            type_name = label_encoder.inverse_transform([i])[0]
            print(f"  {type_name}: {prob*100:.1f}%")
        
        return predicted_type[0], confidence[0]
    
    # টেস্ট প্রেডিকশন
    print("\nTesting prediction on first training image...")
    if len(X_train) > 0:
        test_img = X_train[0:1]  # প্রথম ট্রেনিং ইমেজ
        test_pred = model.predict(test_img, verbose=0)
        true_label = y_train[0]
        true_type = label_encoder.inverse_transform([true_label])[0]
        pred_type = label_encoder.inverse_transform([np.argmax(test_pred)])[0]
        confidence = np.max(test_pred)
        
        print(f"True: {true_type}, Predicted: {pred_type} ({confidence*100:.1f}%)")
    
    print("\nPrediction function ready!")
    print("Usage: predict_pokemon_type_simple('image_path.png', model, label_encoder)")

print(f"\nTotal missing images: {len(missing_images)}")

Loading images...
Successfully loaded 37 images
Missing 406 images
Final dataset: 37 images

Class distribution:
  Water: 8 samples
  Fire: 4 samples
  Psychic: 4 samples
  Grass: 8 samples
  Electric: 2 samples
  Normal: 11 samples

Training set: (29, 128, 128, 3)
Test set: (8, 128, 128, 3)


C:\Users\ascom\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │         401,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             390 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 416,198 (1.59 MB)

 Trainable params: 416,198 (1.59 MB)

 Non-trainable params: 0 (0.00 B)


Training simple model...
Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 607ms/step - accuracy: 0.1379 - loss: 1.8464 - val_accuracy: 0.2500 - val_loss: 1.8060
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 348ms/step - accuracy: 0.1379 - loss: 1.7777 - val_accuracy: 0.2500 - val_loss: 1.7691
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 446ms/step - accuracy: 0.5172 - loss: 1.6723 - val_accuracy: 0.3750 - val_loss: 1.7439
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 557ms/step - accuracy: 0.3793 - loss: 1.6477 - val_accuracy: 0.3750 - val_loss: 1.7268
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 281ms/step - accuracy: 0.4138 - loss: 1.6087 - val_accuracy: 0.3750 - val_loss: 1.7140
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 593ms/step - accuracy: 0.4483 - loss: 1.5803 - val_accuracy: 0.3750 - val_loss: 1.7082
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 474ms/step - accuracy: 0.4483 - loss: 1.5000 - val_accuracy: 0.3750 - val_loss: 1.7048
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 328ms/step - accuracy: 0.3793 - loss: 1.4676 - val_ac


Test Accuracy: 0.5000
Test Loss: 1.7280
Simple model saved!

Testing prediction on first training image...
True: Electric, Predicted: Grass (24.6%)

Prediction function ready!
Usage: predict_pokemon_type_simple('image_path.png', model, label_encoder)

Total missing images: 406
